In [23]:
!pip install Flask Flask-SQLAlchemy Flask-SocketIO


In [30]:
from flask import Flask, render_template, request, redirect, jsonify
from flask_sqlalchemy import SQLAlchemy
from flask_socketio import SocketIO, send
import threading
import time


In [31]:
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///medical_store.db'
db = SQLAlchemy(app)
socketio = SocketIO(app)

class Product(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)
    stock = db.Column(db.Integer, nullable=False)
    price = db.Column(db.Float, nullable=False)

class Customer(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)
    email = db.Column(db.String(100), nullable=False)

# Use application context to create the tables
with app.app_context():
    db.create_all()


In [32]:
@app.route('/')
def index():
    return "Welcome to the Medical Store Management System"

@app.route('/inventory')
def show_inventory():
    products = Product.query.all()
    return jsonify([{'id': p.id, 'name': p.name, 'stock': p.stock, 'price': p.price} for p in products])

@app.route('/add_product', methods=['POST'])
def add_product():
    data = request.json
    new_product = Product(name=data['name'], stock=data['stock'], price=data['price'])
    db.session.add(new_product)
    db.session.commit()
    return "Product added successfully!"


In [33]:
@socketio.on('connect')
def handle_connect():
    print('Client connected')

@socketio.on('disconnect')
def handle_disconnect():
    print('Client disconnected')

@app.route('/alert_stock', methods=['POST'])
def alert_stock():
    data = request.json
    socketio.emit('stock_alert', data)
    return "Stock alert sent!"


In [35]:
def run_app():
    socketio.run(app, host='0.0.0.0', port=5001, allow_unsafe_werkzeug=True)

threading.Thread(target=run_app).start()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://172.28.0.12:5001
INFO:werkzeug:Press CTRL+C to quit
